In [1]:
# python 3.9.16

#

In [2]:
# chunk_size=1000chunk_overlap=200
# k=4(default)
# 不用LLM

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.215
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [2]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
loader = DirectoryLoader('./chinese_pdf', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()


In [4]:
len(documents)

24

In [8]:
documents[0]

Document(page_content=' \n \n      \n保修管理 -厂家SOP \n \n \n2022/11/07                                                  WM.1.0     \n 2023/03/17                                                  WM.2.0 \n目录 \n基本资料  ................................ ................................ ................................ ................................ ... 3 \n1. 角色管理  ................................ ................................ ................................ ..........................  3 \n2. 账号管理  ................................ ................................ ................................ ..........................  3 \n客户管理  ................................ ................................ ................................ ................................ ... 5 \n1. 邀请客户  ................................ ................................ ................................ ..........................  5 \n2. 关联客户  ................................ ................................ ........

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [6]:
len(texts)

27

In [7]:
texts[0]

Document(page_content='保修管理 -厂家SOP \n \n \n2022/11/07                                                  WM.1.0     \n 2023/03/17                                                  WM.2.0 \n目录 \n基本资料  ................................ ................................ ................................ ................................ ... 3 \n1. 角色管理  ................................ ................................ ................................ ..........................  3 \n2. 账号管理  ................................ ................................ ................................ ..........................  3 \n客户管理  ................................ ................................ ................................ ................................ ... 5 \n1. 邀请客户  ................................ ................................ ................................ ..........................  5', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 0})

In [13]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [14]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="GanymedeNil/text2vec-large-chinese",
    model_kwargs={"device": "cuda"}
    # local_files_only=True
)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.
2023-08-31 11:17:26.183535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 11:17:27.120392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
persist_directory = 'db_cn'
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)

In [16]:
vectordb.persist()

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Aug 31 11:18:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [15]:
vectordb = None

In [18]:
q2 = "在哪里可以设定机种?"

In [20]:
retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)

In [21]:
retriever.get_relevant_documents(q2)

/usr/local/lib/python3.10/site-packages/langchain/vectorstores/base.py:256: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}), -340.9338618640956), (Document(page_content='选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码\n与客户的绑定 ; \n \n \n保修设定  \n1. 定义产品 (整机料号信息 ) \n单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代\n料号等信息  \n1.1 机种管理  \n若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . \na. 单笔新增机种  \n厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写\n机种名称 , 点击【保存】完成 ;', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 10}), -360.7415697286397), (Document(page_content='1.9 产品与附件绑定  \n本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . \n厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾\n选对应附 件、填写数量 , 点击【绑定】完成 ; \n \n \n2. 定义部件 （备品料号信息）  \n若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  \n2.1 新增部件机种  \n点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批\n量新增机

[]

In [34]:
vectordb.similarity_search_with_relevance_scores(q2, k=3)

/usr/local/lib/python3.10/site-packages/langchain/vectorstores/base.py:256: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}), -340.9338618640956), (Document(page_content='选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码\n与客户的绑定 ; \n \n \n保修设定  \n1. 定义产品 (整机料号信息 ) \n单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代\n料号等信息  \n1.1 机种管理  \n若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . \na. 单笔新增机种  \n厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写\n机种名称 , 点击【保存】完成 ;', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 10}), -360.7415697286397), (Document(page_content='1.9 产品与附件绑定  \n本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . \n厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾\n选对应附 件、填写数量 , 点击【绑定】完成 ; \n \n \n2. 定义部件 （备品料号信息）  \n若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  \n2.1 新增部件机种  \n点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批\n量新增机

[(Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}),
  -340.9338618640956),
 (Document(page_content='选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码\n与客户的绑定 ; \n \n \n保修设定  \n1. 定义产品 (整机料号信息 ) \n单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代\n料号等信息  \n1.1 机种管理  \n若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . \na. 单笔新增机种  \n厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写\n机种名称 , 点击【保存】完成 ;', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 10}),
  -360.7415697286397),
 (Document(page_content='1.9 产品与附件绑定  \n本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . \n厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾\n选对应附 件、填写数量 , 点击【绑定】完成 ; \n \n \n2. 定义部件 （备品料号信息）  \n若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  \n2.1 新增部件机种  \n点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批\n量新增机种 ;  \n2.2 新增部件  \n点击【新增部件】单笔新增部件 ; 点击【上传部件】下拉框 , 下载模板批量新增部件 ; \n2.3 新增部件客户料号 ; \n点击【料号对照】添加客户料号 , 选定产品 , 点击单笔添加 ; 点击【料号对照】下载模\n板

In [22]:
retriever_k = vectordb.as_retriever(
    search_kwargs={'k': 3}
)

In [30]:
retriever_k.vectorstore

In [25]:
ans2 = retriever_k.get_relevant_documents(q2)

In [28]:
len(ans2)

3

In [24]:
def generate_from_template(context, query):
  return  f"""使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  問題: {query}
  答案:"""

In [27]:

content_with_ans2 = ""
for i, a in enumerate(ans2):
  content_with_ans2+=a.page_content

prompt2 = generate_from_template(content_with_ans2, q2)
print(prompt2)


使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , 
完成该产品序列号判定逻辑设定 .选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码
与客户的绑定 ; 
 
 
保修设定  
1. 定义产品 (整机料号信息 ) 
单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代
料号等信息  
1.1 机种管理  
若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . 
a. 单笔新增机种  
厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写
机种名称 , 点击【保存】完成 ;1.9 产品与附件绑定  
本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . 
厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾
选对应附 件、填写数量 , 点击【绑定】完成 ; 
 
 
2. 定义部件 （备品料号信息）  
若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  
2.1 新增部件机种  
点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批
量新增机种 ;  
2.2 新增部件  
点击【新增部件】单笔新增部件 ; 点击【上传部件】下拉框 , 下载模板批量新增部件 ; 
2.3 新增部件客户料号 ; 
点击【料号对照】添加客户料号 , 选定产品 , 点击单笔添加 ; 点击【料号对照】下载模
板, 批量上传料号 ; 
2.3 新增供应商料号  
点击【供应商料号】 , 选定产品 , 点击按钮 , 点击【添加供应商】 , 添加供应商及供应
商料号 , 【保存】完成 .

  問題: 在哪里可以设定机种?
  答案:


In [35]:
q1 = "如何设定判定逻辑?"

In [36]:
ans1 = retriever_k.get_relevant_documents(q1)

In [37]:

content_with_ans1 = ""
for i, a in enumerate(ans1):
  content_with_ans1+=a.page_content

prompt1 = generate_from_template(content_with_ans1, q1)
print(prompt1)


使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , 
完成该产品序列号判定逻辑设定 .2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .基础设定  
1. 判定命名  
厂家保修管理 后台->基础设定 ->判定命名 ,依用户习惯 , 自主设定相关栏位名称 , 设定
后, 系统相关栏位均以设定名称显示 . 
 
2. 判定逻辑  
设定PPID/SN的逻辑规则 , 用于解析保修起点日期等信息 . 
2.1 添加/编辑判定逻辑  
厂家保修管理 后台->基础设定 ->判定逻辑 , 点击【添加】按钮 , 添加判定逻辑 ;选定判
定逻辑, 点击对该判定逻辑进行编辑 ; 
 
填写解码规则名称、说明、 SN样例；点击【编辑】按钮 进行应用 ; 
 
填写 PPID/SN子串区段 , 依照子串区段 的代表意义 , 可自定义规则  
如下图中点击【月份 规则自定义】 ，  进入自定义模块后 , 点击【新增码值】 , 可自定义
月份解析规则 , 例如 , 可依 PPID/SN规则设定 A代表 10月, B代表 11月份 , C代表 12月
份; 
【添加一项自定义规则】可添加 PPID/SN子串逻辑自定义规则 .

  問題: 如何设定判定逻辑?
  答案:


In [38]:
q3 = "平台可以自动核准保修申请吗?"

In [39]:
ans3 = retriever_k.get_relevant_documents(q3)

In [40]:

content_with_ans3 = ""
for i, a in enumerate(ans3):
  content_with_ans3+=a.page_content

prompt3 = generate_from_template(content_with_ans3, q3)
print(prompt3)


使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  点击【上传附件】下拉框 , 选择【下载模板】填写附件相关信息 , 【批量上传】将编辑好
的文档上传 , 完成批量新增附件 . 
 
保修作业  
1. 保修申请 （手动核准） 
若厂家在客户清单中 , 对应客户已设置【自动核准】功能 ; 客户进行保修申请后 , 平台
依保修条件进行自动核准并分配 ware no.; 若未设定自动核准 , 即可通过手动核准进行
作业 ; 手动核准作业如下 : 
客户进行保修申请后 , 厂家后台保修申请方有数据 ; 
厂家保修管理 后台 ->保修作业 ->保修申请 , 选择对应送修项目 , 点击按钮进入送修信息 ; 
 
点击可查看客户信息 , 点击【接受】进行核准 , 点击【拒绝】进行驳回 ， 需填写拒绝
原因; 当客户送修系统判定不可接受时 (即不符合厂家保修设定 ), 点击可变更核准状
态, 选择保修设定后接受或者拒绝 ;3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .

  問題: 平台可以自动核准保修申请吗?
  答案:


In [41]:
q4 = "自动核准后还需要进行单号提交吗?"

In [44]:
ans4 = retriever_k.get_relevant_documents(q4)

In [45]:

content_with_ans4 = ""
for i, a in enumerate(ans4):
  content_with_ans4+=a.page_content

prompt4 = generate_from_template(content_with_ans4, q4)
print(prompt4)


使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  2. 单号分配  
厂家保修管理 后台->保修作业 ->单号分配 , 勾选送修项目 , 点击【分配 Ware no. 】分
配单号,同客户同申请人的送修清单 , 可一次分配 Ware no.;  
 
勾选送修项目 , 点击【指派 Ware no.】, 系统根据产品型号、退运原因、维修地址、服
务类型等对产品分类并分配  Ware no.;  
备注 : 若客户需使用自主 RMA no.，亦可在【自定单号】添加客户指定 RMA no.点击【上传附件】下拉框 , 选择【下载模板】填写附件相关信息 , 【批量上传】将编辑好
的文档上传 , 完成批量新增附件 . 
 
保修作业  
1. 保修申请 （手动核准） 
若厂家在客户清单中 , 对应客户已设置【自动核准】功能 ; 客户进行保修申请后 , 平台
依保修条件进行自动核准并分配 ware no.; 若未设定自动核准 , 即可通过手动核准进行
作业 ; 手动核准作业如下 : 
客户进行保修申请后 , 厂家后台保修申请方有数据 ; 
厂家保修管理 后台 ->保修作业 ->保修申请 , 选择对应送修项目 , 点击按钮进入送修信息 ; 
 
点击可查看客户信息 , 点击【接受】进行核准 , 点击【拒绝】进行驳回 ， 需填写拒绝
原因; 当客户送修系统判定不可接受时 (即不符合厂家保修设定 ), 点击可变更核准状
态, 选择保修设定后接受或者拒绝 ;3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .

  問題: 自动核准后还需要进行单号提交吗?
  答案:


In [46]:
q5 = "怎么查看保修进度?"

In [47]:
ans5 = retriever_k.get_relevant_documents(q5)

In [48]:

content_with_ans5 = ""
for i, a in enumerate(ans5):
  content_with_ans5+=a.page_content

prompt5 = generate_from_template(content_with_ans5, q5)
print(prompt5)


使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .3. 产品保修 （维修服务 , “退运流程”设置完成后设置）  
厂家后台 ->保修设定 ->产品保修 , 选定对应客户 , 点击按钮进入页面 ; 点击【新增】设
置产品保修 ; 
 
 
点击【选择类别】 , 选择维修 /翻新保修约定 , 选择预设置产品所对应的产品类别 , 选
择保修期间服务内容、具体服务流程、保修期间时长等信息 , 填写完成点击【下一
步】; 
备注: 保修期间服务内容 , 为上章基础设定中 【保修类别】中设定之 信息; 【服务流
程】为送修至服务中心 , 本次需选择；  【服务地址 】为送修至厂家 , 不经服务中心 , 
本次不选择 ; 
 
添加产品 PN, 点击【批量添加 PN】， 带出相同类别 PN产品, 勾选后确认 , 即完成产
品保修设定2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .

  問題: 怎么查看保修进度?
  答案:
